In [1]:
# import libraries
import geopandas as gpd
import pydeck as pdk
import os
from dotenv import load_dotenv
load_dotenv() # loads .env file from main folder for Mapbox API key and local project path

/opt/homebrew/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.12.0-CAPI-1.18.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


True

In [2]:
# load parcel and building data from project folder (Dropbox)
df_parcels = gpd.read_file(os.environ.get('LOCAL_PATH')+"Data/Parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")
# df_buildings = gpd.read_file(os.environ.get('LOCAL_PATH')+"Data/Building Footprints/NYC/geo_export_a80ea1a2-e8e0-4ffd-862c-1199433ac303.shp")

In [ ]:
# join dataframes function
def join_parcels_buildings(parcels, buildings):
    """Join parcels and buildings dataframes."""
    parcels_buildings = buildings.sjoin(parcels, how="left", predicate="within")
    return parcels_buildings

In [ ]:
# set CRS to web mercator
df_parcels = df_parcels.to_crs(epsg=3857)
df_buildings = df_buildings.to_crs(epsg=3857)

In [ ]:
# perform join
df_parcels_buildings = join_parcels_buildings(df_parcels, df_buildings)

In [ ]:
# subset the data frames based on a buffer
def subset(df, df_buildings, index, distance = 75):
    selected_feature = df.loc[index]
    geometry_buffer = selected_feature.geometry.buffer(distance)
    geometry_bounds = selected_feature.geometry.buffer(distance-70)

    return df[df.within(geometry_buffer)], df_buildings[df_buildings.within(geometry_buffer)], geometry_bounds.bounds

In [ ]:
parcels = pdk.Layer(
    "HeatmapLayer",
    data=data,
    opacity=0.9,
    get_position=["lng", "lat"],
    aggregation=pdk.types.String("MEAN"),
    color_range=COLOR_BREWER_BLUE_SCALE,
    threshold=1,
    get_weight="weight",
    pickable=True,
)

heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=data,
    get_position=["lon", "lat"],
    get_weight=1,  # Assuming equal weight for all points
    radius_pixels=50,  # You can adjust the radius as needed
    opacity=0.9,
)

In [3]:
# set CRS to WGS 84
df_parcels = df_parcels.to_crs(epsg=4326)
# df_buildings = df_buildings.to_crs(epsg=4326)

In [4]:
# extract lat and lon from the centerpoint
df_parcels['center_point'] = df_parcels['geometry'].centroid
df_parcels['lon'] = df_parcels['center_point'].x
df_parcels['lat'] = df_parcels['center_point'].y

/var/folders/9g/r0ctqhfj26l910sgbwcdndq00000gn/T/ipykernel_8474/1599866429.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_parcels['center_point'] = df_parcels['geometry'].centroid


In [ ]:
# view df
print(df_parcels['geometry'].head())
print(df_parcels['center_point'].head())
print(df_parcels['center_point'].head())
print(df_parcels['lon'].head())
print(df_parcels['lat'].head())

In [ ]:
# Create a PyDeck layer
layer = pdk.Layer(
    'ScatterplotLayer',
    data=df_parcels,
    get_position='[lon, lat]',
    get_radius=10,
    get_fill_color=[255, 0, 0],
    auto_highlight=True,
    pickable=True
)

# Create the map
view_state = pdk.ViewState(
    latitude=40.76,
    longitude=-74,
    zoom=11,
    max_zoom=12,
    min_zoom=10
)

r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state
)

# Display the visualization in the notebook (warning: increases notebook size significantly)
# r.show()

# save the visualization to an HTML file (git-ignored)
r.to_html('parcels-map.html')